In [1]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(version = "3.19")
BiocManager::install("EDASeq")
BiocManager::install("limma")
BiocManager::install("edgeR")
BiocManager::install("TCGAbiolinks")
BiocManager::install("sesame")
BiocManager::install("SummarizedExperiment", force = TRUE)

BiocManager::install("dplyr", force = TRUE)
install.packages('gplots')
install.packages("pheatmap")
install.packages("SummarizedExperiment")
install.packages("ggrepel")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.25), R 4.4.1 (2024-06-14)

Installing package(s) 'BiocVersion'

Old packages: 'askpass', 'commonmark', 'credentials', 'sys', 'xfun'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.25), R 4.4.1 (2024-06-14)

Installing package(s) 'EDASeq'

also installing the dependencies ‘formatR’, ‘abind’, ‘SparseArray’, ‘lambda.r’, ‘futile.options’, ‘MatrixGenerics’, ‘S4Arrays’, ‘DelayedArray’, ‘deldir’, ‘RcppEigen’, ‘RCurl’, ‘rjson’, ‘futile.logger’, ‘snow’, ‘BH’, ‘SummarizedExperiment’, ‘UCSC.utils

In [2]:
# Load libraries
library(TCGAbiolinks)
library(SummarizedExperiment)
library(dplyr)
library(sesame)
library(edgeR)
library(EDASeq)
library(limma)
library(gplots)
library(pheatmap)
library(ggrepel)

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges, rowRanks,
    rowSdDiffs, rowSds, rowSums2, ro

###**Glioma is a growth of cells that starts in the brain or spinal cord.**

In [3]:
#get project summary of TCGA-LGG dataset number of samples n=516
summary <- getProjectSummary("TCGA-LGG")

In [4]:
summary$data_categories$data_category

[1] "Simple Nucleotide Variation"  "Sequencing Reads"            
 [3] "Biospecimen"                  "Clinical"                    
 [5] "Copy Number Variation"        "Transcriptome Profiling"     
 [7] "DNA Methylation"              "Proteome Profiling"          
 [9] "Somatic Structural Variation" "Structural Variation"

### **Download Gene expression data for DGA between IDH statuses**

In [5]:
# Query for RNA-seq gene expression data for LGG
query_DGE_TCGA <- GDCquery(project = "TCGA-LGG",
                       data.category = "Transcriptome Profiling",
                       experimental.strategy = "RNA-Seq",
                       workflow.type = "STAR - Counts",
                       access = "open")

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-LGG

--------------------

oo Filtering results

--------------------

ooo By access

ooo By experimental.strategy

ooo By workflow.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------



In [6]:
#get results of the query
output_query_DGE_TCGA <- getResults(query_DGE_TCGA)

#generate DGE metadata
DGE_metadata <- data.frame("sample_names"=output_query_DGE_TCGA$cases,"sample_type"=output_query_DGE_TCGA$sample_type)

#write metadata to file
write.csv(DGE_metadata,"DGE_metadata.csv")

head(DGE_metadata)

,sample_names,sample_type
,<chr>,<chr>
1,TCGA-E1-A7YQ-01A-11R-A34R-07,Primary Tumor
2,TCGA-S9-A7QW-01A-11R-A34F-07,Primary Tumor
3,TCGA-FG-A4MU-01B-11R-A28M-07,Primary Tumor
4,TCGA-QH-A6CU-01A-11R-A31N-07,Primary Tumor
5,TCGA-DU-8162-01A-21R-2256-07,Primary Tumor
6,TCGA-S9-A7R8-01A-11R-A34R-07,Primary Tumor


In [7]:
# Download the gene expression data
GDCdownload(query_DGE_TCGA)


GDCdownload will download 534 files. A total of 2.267094301 GB

The total size of files is big. We will download files in chunks




Downloading: 250 MB     

Downloading: 250 MB     

Downloading: 67 MB     

In [8]:
# Prepare the gene expression data
expression_data <- GDCprepare(query_DGE_TCGA)

DEA_matrix <- assay(expression_data, "unstranded")



|====================================================|100%                      Completed after 24 s 


Starting to add information to samples

 => Add clinical information to samples

 => Adding TCGA molecular information from marker papers

 => Information will have prefix 'paper_' 

lgg subtype information from:doi:10.1016/j.cell.2015.12.028

Available assays in SummarizedExperiment : 
  => unstranded
  => stranded_first
  => stranded_second
  => tpm_unstrand
  => fpkm_unstrand
  => fpkm_uq_unstrand



In [13]:
write.csv(DEA_matrix, "LGG_rawdata_DGE.csv")

In [9]:
expression_data$paper_IDH.status

[1] WT     Mutant WT     Mutant WT     Mutant Mutant Mutant Mutant Mutant
 [11] Mutant WT     Mutant Mutant Mutant Mutant Mutant Mutant WT     Mutant
 [21] Mutant Mutant WT     WT     Mutant WT     Mutant Mutant WT     Mutant
 [31] Mutant Mutant Mutant Mutant WT     Mutant Mutant WT     WT     Mutant
 [41] Mutant WT     Mutant Mutant WT     Mutant WT     Mutant WT     Mutant
 [51] WT     WT     Mutant Mutant Mutant Mutant WT     <NA>   Mutant Mutant
 [61] Mutant WT     Mutant Mutant WT     Mutant Mutant <NA>   Mutant WT    
 [71] Mutant Mutant Mutant Mutant Mutant Mutant WT     Mutant Mutant Mutant
 [81] Mutant Mutant Mutant WT     WT     Mutant Mutant Mutant <NA>   Mutant
 [91] Mutant Mutant WT     Mutant Mutant WT     Mutant WT     Mutant Mutant
[101] Mutant Mutant WT     WT     Mutant Mutant Mutant Mutant Mutant Mutant
[111] Mutant Mutant WT     Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[121] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[131] Mutant Mutant Mutant Mutant Mutant WT     Mutant Mutant WT     <NA>  
[141] <NA>   Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant WT    
[151] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[161] Mutant Mutant WT     Mutant Mutant Mutant Mutant WT     Mutant Mutant
[171] Mutant <NA>   <NA>   Mutant Mutant Mutant WT     Mutant Mutant Mutant
[181] WT     Mutant WT     WT     Mutant Mutant WT     WT     Mutant Mutant
[191] Mutant Mutant Mutant Mutant Mutant Mutant Mutant WT     Mutant Mutant
[201] WT     Mutant Mutant WT     Mutant Mutant Mutant Mutant Mutant <NA>  
[211] WT     Mutant <NA>   Mutant WT     Mutant Mutant Mutant Mutant Mutant
[221] Mutant Mutant Mutant Mutant Mutant WT     Mutant Mutant Mutant Mutant
[231] <NA>   WT     Mutant Mutant Mutant Mutant WT     <NA>   Mutant <NA>  
[241] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[251] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[261] Mutant Mutant WT     <NA>   WT     Mutant Mutant Mutant Mutant <NA>  
[271] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant <NA>  
[281] Mutant WT     Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[291] Mutant Mutant WT     Mutant <NA>   Mutant Mutant WT     Mutant Mutant
[301] Mutant Mutant WT     Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[311] Mutant Mutant Mutant WT     Mutant Mutant Mutant <NA>   Mutant Mutant
[321] Mutant WT     Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[331] WT     Mutant Mutant Mutant Mutant <NA>   WT     Mutant Mutant Mutant
[341] <NA>   Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant <NA>  
[351] Mutant Mutant Mutant WT     Mutant Mutant Mutant WT     Mutant Mutant
[361] Mutant WT     Mutant Mutant Mutant Mutant Mutant Mutant WT     Mutant
[371] Mutant WT     Mutant Mutant Mutant Mutant Mutant Mutant WT     Mutant
[381] Mutant Mutant Mutant Mutant Mutant Mutant WT     WT     Mutant Mutant
[391] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[401] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[411] WT     Mutant Mutant Mutant Mutant Mutant Mutant Mutant WT     WT    
[421] Mutant Mutant Mutant Mutant Mutant WT     Mutant WT     Mutant Mutant
[431] Mutant Mutant Mutant Mutant Mutant Mutant Mutant WT     Mutant Mutant
[441] Mutant Mutant Mutant Mutant Mutant Mutant WT     WT     WT     Mutant
[451] Mutant Mutant Mutant Mutant Mutant WT     WT     Mutant Mutant Mutant
[461] Mutant WT     Mutant WT     WT     WT     Mutant Mutant Mutant Mutant
[471] Mutant Mutant Mutant WT     Mutant Mutant Mutant Mutant Mutant Mutant
[481] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant
[491] Mutant Mutant Mutant WT     WT     WT     Mutant Mutant <NA>   Mutant
[501] Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant Mutant WT    
[511] Mutant Mutant WT     WT     Mutant WT     Mutant Mutant Mutant Mutant
[521] Mutant Mutant Mutant Mutant Mutant Mutant WT

**Normalize data based on lowly expressed genes**

In [10]:
# Normalize data with TCGAanalyze_Normalization
DEA_matrix_norm <- TCGAanalyze_Normalization(
  tabDF = DEA_matrix,
  geneInfo = geneInfoHT,
  method = "geneLength"
)


I Need about  384 seconds for this Complete Normalization Upper Quantile  [Processing 80k elements /s]  

Step 1 of 4: newSeqExpressionSet ...

Step 2 of 4: withinLaneNormalization ...

Step 3 of 4: betweenLaneNormalization ...

Step 4 of 4: exprs ...



**Filter normalized data by checking quantile cutoff**

In [11]:
# filter coad data
DEA_matrix_filter <- TCGAanalyze_Filtering(
  tabDF = DEA_matrix_norm,
  method = "quantile",
  qnt.cut = 0.25
)




**Transform normalized and filtered data into Log2**

In [12]:
DEA_matrix_log2 <- log2(DEA_matrix_filter + 1)
write.csv(DEA_matrix_log2, "DGE_matrix_log2.csv")

In [14]:
head(DEA_matrix_log2)

,TCGA-E1-A7YQ-01A-11R-A34R-07,TCGA-S9-A7QW-01A-11R-A34F-07,TCGA-FG-A4MU-01B-11R-A28M-07,TCGA-QH-A6CU-01A-11R-A31N-07,TCGA-DU-8162-01A-21R-2256-07,TCGA-S9-A7R8-01A-11R-A34R-07,TCGA-DB-A4XG-01A-11R-A27Q-07,TCGA-E1-A7YY-01A-11R-A34R-07,TCGA-RY-A840-01A-11R-A36H-07,TCGA-S9-A7QZ-01A-12R-A34R-07,⋯,TCGA-HT-A615-01A-11R-A29R-07,TCGA-HT-7468-01A-11R-2027-07,TCGA-HT-A4DS-01A-11R-A26U-07,TCGA-HT-8107-01A-13R-2404-07,TCGA-HT-A5R5-01A-11R-A28M-07,TCGA-HT-7860-01A-11R-2403-07,TCGA-TM-A84H-01A-11R-A36H-07,TCGA-HT-7681-01A-11R-2403-07,TCGA-DB-5270-01A-02R-1470-07,TCGA-IK-8125-01A-11R-2256-07
ENSG00000000003,11.533816,11.411511,12.211888,11.778488,10.680360,10.805744,11.861475,9.413628,11.154185,11.144658,⋯,11.349834,12.294046,10.032046,9.590587,11.661778,12.270879,12.130249,11.424166,11.598518,11.793197
ENSG00000000005,2.584963,1.584963,4.321928,2.807355,4.169925,3.459432,2.000000,2.807355,1.000000,3.321928,⋯,3.169925,3.459432,0.000000,3.169925,2.321928,1.000000,1.584963,3.321928,5.727920,3.807355
ENSG00000000419,10.497852,9.842350,10.688250,9.679480,10.256209,9.710806,10.326429,10.122828,9.891784,10.002815,⋯,10.032046,10.324181,9.372865,10.136991,9.413628,10.269127,10.363040,9.967226,9.949827,10.360847
ENSG00000000457,10.074141,8.654636,9.084808,8.417853,9.623881,8.891784,9.383704,9.098032,8.861087,9.537218,⋯,9.209453,9.930737,7.994353,9.346514,8.573647,9.575539,9.778077,8.965784,9.144658,9.864186
ENSG00000000460,9.011227,7.366322,8.422065,6.321928,7.781360,8.224002,7.515700,6.918863,7.022368,7.781360,⋯,7.754888,8.027906,6.209453,6.781360,6.906891,8.566054,8.022368,6.727920,7.499846,8.134426
ENSG00000000938,10.651052,6.714246,9.204571,8.005625,8.672425,9.449149,8.430453,8.335390,7.339850,9.315150,⋯,8.247928,8.569856,8.409391,9.207014,9.842350,9.729621,7.159871,7.614710,8.011227,8.974415


### **Save all objects into RDS files for future references**

### **Download Mulecular subtypes to get IDH Status:**

In [16]:
# get molecular subtype for LGG data
lgg.gbm.subtype <- TCGAquery_subtype(tumor = "lgg")

colnames(lgg.gbm.subtype)

lgg subtype information from:doi:10.1016/j.cell.2015.12.028



[1] "patient"                                      
 [2] "Tissue.source.site"                           
 [3] "Study"                                        
 [4] "BCR"                                          
 [5] "Whole.exome"                                  
 [6] "Whole.genome"                                 
 [7] "RNAseq"                                       
 [8] "SNP6"                                         
 [9] "U133a"                                        
[10] "HM450"                                        
[11] "HM27"                                         
[12] "RPPA"                                         
[13] "Histology"                                    
[14] "Grade"                                        
[15] "Age..years.at.diagnosis."                     
[16] "Gender"                                       
[17] "Survival..months."                            
[18] "Vital.status..1.dead."                        
[19] "Karnofsky.Performance.Score"                  
[20] "Mutation.Count"                               
[21] "Percent.aneuploidy"                           
[22] "IDH.status"                                   
[23] "X1p.19q.codeletion"                           
[24] "IDH.codel.subtype"                            
[25] "MGMT.promoter.status"                         
[26] "Chr.7.gain.Chr.10.loss"                       
[27] "Chr.19.20.co.gain"                            
[28] "TERT.promoter.status"                         
[29] "TERT.expression..log2."                       
[30] "TERT.expression.status"                       
[31] "ATRX.status"                                  
[32] "DAXX.status"                                  
[33] "Telomere.Maintenance"                         
[34] "BRAF.V600E.status"                            
[35] "BRAF.KIAA1549.fusion"                         
[36] "ABSOLUTE.purity"                              
[37] "ABSOLUTE.ploidy"                              
[38] "ESTIMATE.stromal.score"                       
[39] "ESTIMATE.immune.score"                        
[40] "ESTIMATE.combined.score"                      
[41] "Original.Subtype"                             
[42] "Transcriptome.Subtype"                        
[43] "Pan.Glioma.RNA.Expression.Cluster"            
[44] "IDH.specific.RNA.Expression.Cluster"          
[45] "Pan.Glioma.DNA.Methylation.Cluster"           
[46] "IDH.specific.DNA.Methylation.Cluster"         
[47] "Supervised.DNA.Methylation.Cluster"           
[48] "Random.Forest.Sturm.Cluster"                  
[49] "RPPA.cluster"                                 
[50] "Telomere.length.estimate.in.blood.normal..Kb."
[51] "Telomere.length.estimate.in.tumor..Kb."

**Select specific columns from lgg subtype data and make a new dataframe**

In [17]:
lgg_subtype_df <- data.frame(
                            patient = as.character(lgg.gbm.subtype$patient),
                            IDH.status= as.character(lgg.gbm.subtype$IDH.status),
                            IDH.codel.subtype= as.character(lgg.gbm.subtype$IDH.codel.subtype),
                            IDH.specific.DNA.Methylation.Cluster= as.character(lgg.gbm.subtype$IDH.specific.DNA.Methylation.Cluster),
                            IDH.specific.RNA.Expression.Cluster=as.character(lgg.gbm.subtype$IDH.specific.RNA.Expression.Cluster))

head(lgg_subtype_df)

,patient,IDH.status,IDH.codel.subtype,IDH.specific.DNA.Methylation.Cluster,IDH.specific.RNA.Expression.Cluster
,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-CS-4938,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
2,TCGA-CS-4941,WT,IDHwt,IDHwt-K2,IDHwt-R2
3,TCGA-CS-4942,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
4,TCGA-CS-4943,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
5,TCGA-CS-4944,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
6,TCGA-CS-5390,Mutant,IDHmut-codel,IDHmut-K3,IDHmut-R1


In [18]:
length(colnames(DEA_matrix_log2))

length(rownames(lgg_subtype_df))

[1] 534

[1] 516

In [19]:
typeof(lgg_subtype_df$patient)

[1] "character"

In [21]:

#extarct sample barcodes from lgg subtypes data
lgg_subtype_samplebarcodes <- lgg_subtype_df$patient

In [22]:

head(DGE_metadata$sample_names)
head(lgg_subtype_samplebarcodes)

[1] "TCGA-E1-A7YQ-01A-11R-A34R-07" "TCGA-S9-A7QW-01A-11R-A34F-07"
[3] "TCGA-FG-A4MU-01B-11R-A28M-07" "TCGA-QH-A6CU-01A-11R-A31N-07"
[5] "TCGA-DU-8162-01A-21R-2256-07" "TCGA-S9-A7R8-01A-11R-A34R-07"

[1] "TCGA-CS-4938" "TCGA-CS-4941" "TCGA-CS-4942" "TCGA-CS-4943" "TCGA-CS-4944"
[6] "TCGA-CS-5390"

**Since sample barcodes formats are different in methylation, DGE and subtype data, removing last part and keeping only the
first 12 digits TCGA-TQ-A7RH of patientIDs**

In [23]:
# Extract the first 12 characters (sample identifier) from each barcode
dge_short_barcodes <- substr(DGE_metadata$sample_names, 1, 12)

In [24]:
dge_short_barcodes[1:10]
head(lgg_subtype_samplebarcodes)

[1] "TCGA-E1-A7YQ" "TCGA-S9-A7QW" "TCGA-FG-A4MU" "TCGA-QH-A6CU" "TCGA-DU-8162"
 [6] "TCGA-S9-A7R8" "TCGA-DB-A4XG" "TCGA-E1-A7YY" "TCGA-RY-A840" "TCGA-S9-A7QZ"

[1] "TCGA-CS-4938" "TCGA-CS-4941" "TCGA-CS-4942" "TCGA-CS-4943" "TCGA-CS-4944"
[6] "TCGA-CS-5390"

**Add short barcodes columns in Methylation and DGE metadata**

In [25]:
DGE_metadata$patient <- dge_short_barcodes


In [26]:

head(DGE_metadata)

,sample_names,sample_type,patient
,<chr>,<chr>,<chr>
1,TCGA-E1-A7YQ-01A-11R-A34R-07,Primary Tumor,TCGA-E1-A7YQ
2,TCGA-S9-A7QW-01A-11R-A34F-07,Primary Tumor,TCGA-S9-A7QW
3,TCGA-FG-A4MU-01B-11R-A28M-07,Primary Tumor,TCGA-FG-A4MU
4,TCGA-QH-A6CU-01A-11R-A31N-07,Primary Tumor,TCGA-QH-A6CU
5,TCGA-DU-8162-01A-21R-2256-07,Primary Tumor,TCGA-DU-8162
6,TCGA-S9-A7R8-01A-11R-A34R-07,Primary Tumor,TCGA-S9-A7R8


**Now barcode format is the same so check if they matches**

In [27]:
# Check which short barcodes are present in lgg_subtype barcodes
matched_columns_dge <- DGE_metadata$patient %in% lgg_subtype_df$patient
# Print how many barcodes matched now
print(paste("Number of matched barcodes in DGE data:", sum(matched_columns_dge)))



[1] "Number of matched barcodes in DGE data: 534"


### **Merge DGE Data with IDH status from subtype data**

In [28]:
dge_metadata_with_IDHstatus <- merge(DGE_metadata, lgg_subtype_df, by = "patient", all = TRUE)

head(dge_metadata_with_IDHstatus)

write.csv(dge_metadata_with_IDHstatus, "dge_metadata_with_IDHstatus.csv",row.names = TRUE)

,patient,sample_names,sample_type,IDH.status,IDH.codel.subtype,IDH.specific.DNA.Methylation.Cluster,IDH.specific.RNA.Expression.Cluster
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-CS-4938,TCGA-CS-4938-01B-11R-1896-07,Primary Tumor,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
2,TCGA-CS-4941,TCGA-CS-4941-01A-01R-1470-07,Primary Tumor,WT,IDHwt,IDHwt-K2,IDHwt-R2
3,TCGA-CS-4942,TCGA-CS-4942-01A-01R-1470-07,Primary Tumor,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
4,TCGA-CS-4943,TCGA-CS-4943-01A-01R-1470-07,Primary Tumor,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
5,TCGA-CS-4944,TCGA-CS-4944-01A-01R-1470-07,Primary Tumor,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
6,TCGA-CS-5390,TCGA-CS-5390-01A-02R-1470-07,Primary Tumor,Mutant,IDHmut-codel,IDHmut-K3,IDHmut-R1


### **Differential Gene Expression Analysis of TCGA-LGG data**

**Perform differetial expression analysis using TCGAanalyze_DEA method**

In [30]:
# Remove rows with NA values from metadata in IDH.status column before subsetting
dge_metadata_with_IDHstatus <- dge_metadata_with_IDHstatus[!is.na(dge_metadata_with_IDHstatus$IDH.status), ]

# Separate samples into Mutant and Wildtype groups based on IDH status
mutant_samples <- dge_metadata_with_IDHstatus[dge_metadata_with_IDHstatus$IDH.status == "Mutant", ]
wildtype_samples <- dge_metadata_with_IDHstatus[dge_metadata_with_IDHstatus$IDH.status == "WT", ]

# Find column names in DEA_matrix_log2 that match mutant and wildtype samples
matched_col_mutant <- colnames(DEA_matrix_log2)[colnames(DEA_matrix_log2) %in% mutant_samples$sample_names]
matched_col_wt <- colnames(DEA_matrix_log2)[colnames(DEA_matrix_log2) %in% wildtype_samples$sample_names]

# Extract log2 values for wildtype and mutant samples
log2_wildtype <- DEA_matrix_log2[, matched_col_wt]
log2_mutant <- DEA_matrix_log2[, matched_col_mutant]

# Perform differential gene expression analysis using edgeR
lgg_dge_results <- TCGAanalyze_DEA(
  mat1 = log2_wildtype,       # Wildtype expression matrix
  mat2 = log2_mutant,         # Mutant expression matrix
  Cond1type = "WT",           # Condition label for wildtype
  Cond2type = "Mutant",       # Condition label for mutant
  pipeline = "edgeR",
  fdr.cut = 0.01,
  logFC.cut = 2,
  method = "glmLRT"          # Using glmLRT method for DGE
)

# Analyze the levels of differential expression
lgg_dge_levels <- TCGAanalyze_LevelTab(
  lgg_dge_results,
  "WT",
  "Mutant",
  log2_wildtype,
  log2_mutant
)

# View the first few rows of the differential expression results
head(lgg_dge_levels)

# Save the results to CSV
write.csv(lgg_dge_levels, "LGG_DGE_levels.csv")






Batch correction skipped since no factors provided

----------------------- DEA -------------------------------

o 97 samples in Cond1type WT

o 434 samples in Cond2type Mutant

o 34549 features as miRNA or genes 

This may take some minutes...

----------------------- END DEA -------------------------------



,mRNA,logFC,FDR,Delta,WT,Mutant,start,end
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
ENSG00000257935,ENSG00000257935,2.389878,3.557592e-98,2.089740,0.8744125,5.071961,113472003,113480624
ENSG00000286953,ENSG00000286953,2.279222,1.435709e-80,1.830508,0.8031284,4.345280,97799404,97882687
ENSG00000129152,ENSG00000129152,2.155459,3.568524e-69,1.718389,0.7972266,3.951736,17719571,17722136
ENSG00000163206,ENSG00000163206,2.350968,1.612729e-76,1.597042,0.6793125,3.947077,152878322,152885047
ENSG00000165762,ENSG00000165762,2.093974,3.245439e-57,1.446258,0.6906759,3.330757,19875142,19883932
ENSG00000189295,ENSG00000189295,2.478826,1.136234e-67,1.188988,0.4796576,3.221227,16654066,16675540


In [31]:
# Separate upregulated and downregulated genes based on logFC cutoff
upreg_genes <- lgg_dge_levels[lgg_dge_levels$logFC > 2, ]
dnreg_genes <- lgg_dge_levels[lgg_dge_levels$logFC < -2, ]

# Change column names to "GeneIDs"
colnames(upreg_genes)[1] <- "GeneIDs"
colnames(dnreg_genes)[1] <- "GeneIDs"

# Save upregulated and downregulated genes to CSV files
write.csv(upreg_genes, "LGG_Uregulated_genes.csv")
write.csv(dnreg_genes, "LGG_Downregulated_genes.csv")


In [44]:
volcano_data <- data.frame(
  Gene = rownames(lgg_dge_results),
  logFC = lgg_dge_results$logFC,
  FDR = lgg_dge_results$FDR,
  IDH_status = ifelse(rownames(lgg_dge_results) %in% upreg_genes$GeneIDs, "Mutant",
                      ifelse(rownames(lgg_dge_results) %in% dnreg_genes$GeneIDs, "Wild Type", "Not significant"))
)

# Calculate -log10 FDR for the y-axis
volcano_data$negLogFDR <- -log10(volcano_data$FDR)
png(filename = "Volcano_Heatmap_LGG-TCGA.png", res = 300, width=1500, height=1200)
# Create volcano plot
ggplot(volcano_data, aes(x = logFC, y = negLogFDR, color = IDH_status)) +
  geom_point(size = 2, alpha = 0.6) +
  geom_vline(xintercept = c(-1, 1), linetype = "dashed", color = "black") +
  geom_hline(yintercept = 1.3, linetype = "dashed", color = "black") +
  scale_color_manual(values = c("Mutant" = "red", "Wild Type" = "blue", "Not significant" = "grey")) +
  theme_minimal() +
  labs(x = "Log2 Fold Change", y = "-Log10 FDR", title = "Volcano Plot of WT vs Mut", color = "Sample Type") +
  theme(axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        plot.title = element_text(hjust = 0.5, size = 16),
        legend.position = "top") +
  ggrepel::geom_label_repel(data = subset(volcano_data, abs(logFC) > 2 & FDR < 0.01), aes(label = Gene))

dev.off()

Warning message:
“ggrepel: 66 unlabeled data points (too many overlaps). Consider increasing max.overlaps”


pdf 
  2

In [38]:
write.csv(lgg_dge_results,"LGG_DGE_results.csv")

### **Heatmap generation by sample clustering**

In [40]:
# take data from coad_log2norm matrix with rownames by coad_dge_levels
heatmap_data <- head(DEA_matrix_log2[rownames(lgg_dge_levels),])

In [41]:
# color the plot by kind of mutant
idh_type <- c(rep("Mutant",434), rep("WT",97))

color_code <- c(rep("Mutant",434), rep("WT",97))

In [42]:
color_code <- c()

for(i in idh_type ){
  if(i == "WT"){
    color_code <- c(color_code,"blue")
  }else{
     color_code <- c(color_code,"red")
  }
}

In [ ]:
typeof(color_code)

[1] "character"

In [43]:
# Specify the output file and dimensions for heapmap

png(filename = "SampleCluster_Heatmap_LGG-TCGA.png", res = 300, width=1500, height=1200)

heatmap.2(
            x = as.matrix(heatmap_data),                    # data matrix
            dendrogram = "col", # Creates both row and column dendrograms
            Rowv =F, Colv =T,
            scale = "row",  # Scale by row (normalize each row)
            trace = "none",                  # No trace lines inside the heatmap
            col = bluered(100),              # Color scheme (you can choose another one)
            #margins = c(3, 3),               # Adjust margins
            key = TRUE,                      # Show a color key
            cexRow = 0.5,                    # Font size for row labels
            cexCol = 0.5,    # Font size for column labels
            main = "LGG Sample clusters heatmap" ,
            #ColSideColors = color_code
)

dev.off()

pdf 
  2